In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langgraph langchain_google_genai

In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GOOGLE_API_KEY")

GOOGLE_API_KEY:  ········


## Fan out and Fan In

In [7]:
from IPython.display import Image, display

from typing import Any
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

class State(TypedDict):
    state: str

class ReturnNodeValue:
    def __init__(self, node_secret: str):
        self._value = node_secret

    def __call__(self, state: State) -> Any:
        print(f"Adding {self._value} to {state['state']}")
        return {
            "state": [self._value]
        }

builder = StateGraph(State)

builder.add_node("a", ReturnNodeValue("I 'm A"))
builder.add_node("b", ReturnNodeValue("I 'm B"))
builder.add_node("c", ReturnNodeValue("I 'm C"))
builder.add_node("d", ReturnNodeValue("I 'm D"))

builder.add_edge(START, "a")
builder.add_edge("a", "b")
builder.add_edge("b", "c")
builder.add_edge("c", "d")
builder.add_edge("d", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))


ValueError: Failed to reach https://mermaid.ink API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [8]:
graph.invoke({
    "state": []
})

Adding I 'm A to []
Adding I 'm B to ["I 'm A"]
Adding I 'm C to ["I 'm B"]
Adding I 'm D to ["I 'm C"]


{'state': ["I 'm D"]}

Now lets run B and C parallel

In [9]:

builder = StateGraph(State)

builder.add_node("a", ReturnNodeValue("I 'm A"))
builder.add_node("b", ReturnNodeValue("I 'm B"))
builder.add_node("c", ReturnNodeValue("I 'm C"))
builder.add_node("d", ReturnNodeValue("I 'm D"))

builder.add_edge(START, "a")
builder.add_edge("a", "b")
builder.add_edge("a", "c")
builder.add_edge("b", "d")
builder.add_edge("c", "d")
builder.add_edge("d", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [13]:
from langgraph.errors import InvalidUpdateError

try:
    graph.invoke({
        "state": []
    })
except InvalidUpdateError as e:
    print(f"An error occured {e}")

Adding I 'm A to []
Adding I 'm B to ["I 'm A"]
Adding I 'm C to ["I 'm A"]
An error occured At key 'state': Can receive only one value per step. Use an Annotated key to handle multiple values.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_CONCURRENT_GRAPH_UPDATE
